# To experiment with some deep learning models on the time series data given

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
df = pd.read_csv("./data/processed/Mastercard_stock_history_processed.csv")
df.head()
df = df[["Open", "Volume",  "lag_1","lag_2","MA","M_STD", "month", "day","quarter","Close"]]

In [61]:
train_df = df[:-100]
test_df = df[-100:]

In [ ]:
train_df_torch = torch.from_numpy(train_df.to_numpy()).type(torch.float32)
test_df_torch = torch.from_numpy(test_df.to_numpy()).type(torch.float32)

X_train, y_train = train_df_torch[:, :-1], train_df_torch[:, -1].reshape(-1, 1)
X_test, y_test= test_df_torch[:, :-1], test_df_torch[:, -1].reshape(-1, 1)


In [ ]:
class TimeSeriesLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_rate):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Define the fully connected layer
        self.dense1 = nn.Linear(hidden_size, hidden_size * 2)
        self.dense2 = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.dense3 = nn.Linear(hidden_size * 2, output_size)

        self.relu = nn.ReLU()
        self.elu = nn.ELU(alpha=0.5)
        self.lrelu = nn.LeakyReLU(negative_slope=0.02)
        self.sm = nn.Sigmoid()
        self.tanh = nn.Tanh()
        # add dropout layer
        self.dropout = nn.Dropout(dropout_rate)

        
    
    def forward(self, x):
        # Forward propagate LSTM
        op , _ = self.lstm(x)

        a1 = self.dense1(op)
        a1 = self.lrelu(a1)
        o1 = self.dropout(a1)
        a2 = self.tanh(self.dense2(o1))
        a3 = self.dense3(a2)
        
        # Decode the hidden state of the last time stepout = self.fc(out[:, -1, :])
        # return self.tanh(a3) 
        return a3

In [ ]:
model = TimeSeriesLSTM(
    input_size=9,
    num_layers=5,
    hidden_size=64,
    output_size=1,
    dropout_rate=0.5
)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
# loader = load_data("data/processed/mastercard_stock_history_processed.csv", batch_size=8)
loader = DataLoader(TensorDataset(X_train, y_train), shuffle = False, batch_size = 8)
 
n_epochs = 100
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    if epoch % 1 != 0:
        continue
    model.eval()
    with torch.no_grad():
        y_pred = model(X_train)
        train_rmse = np.sqrt(loss_fn(y_pred, y_train))
        y_pred = model(X_test)
        test_rmse = np.sqrt(loss_fn(y_pred, y_test))
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))

Epoch 0: train RMSE 112.3993, test RMSE 212.5367
Epoch 1: train RMSE 106.5553, test RMSE 228.1010
Epoch 2: train RMSE 117.8843, test RMSE 200.9998
Epoch 3: train RMSE 122.7630, test RMSE 191.9818
Epoch 4: train RMSE 130.9146, test RMSE 178.4600
Epoch 5: train RMSE 138.8247, test RMSE 166.5179
Epoch 6: train RMSE 104.7817, test RMSE 234.3151
Epoch 7: train RMSE 108.2731, test RMSE 222.9565
Epoch 8: train RMSE 116.1377, test RMSE 204.4841
Epoch 9: train RMSE 131.7332, test RMSE 177.1793
Epoch 10: train RMSE 124.7658, test RMSE 188.5098
Epoch 11: train RMSE 127.6381, test RMSE 183.7127
Epoch 12: train RMSE 131.5175, test RMSE 177.5156
Epoch 13: train RMSE 139.8809, test RMSE 164.9871
Epoch 14: train RMSE 131.6366, test RMSE 177.3298
Epoch 15: train RMSE 137.3628, test RMSE 168.6588
Epoch 16: train RMSE 142.8907, test RMSE 160.6932
Epoch 17: train RMSE 150.3483, test RMSE 150.4261
Epoch 18: train RMSE 158.0947, test RMSE 140.2089
Epoch 19: train RMSE 155.1052, test RMSE 144.1053


KeyboardInterrupt: 